# 75 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from xgboost import XGBClassifier

2023-12-01 10:13:25.400596: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 10:13:27.698659: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-01 10:13:27.698783: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-01 10:13:28.052234: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-01 10:13:28.789714: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-01 10:13:28.791039: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
no_of_patients=75

In [3]:
train=sorted(glob('files3/*.edf'))
train=train[:no_of_patients]
train

['files3/S001R05.edf',
 'files3/S002R05.edf',
 'files3/S003R05.edf',
 'files3/S004R05.edf',
 'files3/S005R05.edf',
 'files3/S006R05.edf',
 'files3/S007R05.edf',
 'files3/S008R05.edf',
 'files3/S009R05.edf',
 'files3/S010R05.edf',
 'files3/S011R05.edf',
 'files3/S012R05.edf',
 'files3/S013R05.edf',
 'files3/S014R05.edf',
 'files3/S015R05.edf',
 'files3/S016R05.edf',
 'files3/S017R05.edf',
 'files3/S018R05.edf',
 'files3/S019R05.edf',
 'files3/S020R05.edf',
 'files3/S021R05.edf',
 'files3/S022R05.edf',
 'files3/S023R05.edf',
 'files3/S024R05.edf',
 'files3/S025R05.edf',
 'files3/S026R05.edf',
 'files3/S027R05.edf',
 'files3/S028R05.edf',
 'files3/S029R05.edf',
 'files3/S030R05.edf',
 'files3/S031R05.edf',
 'files3/S032R05.edf',
 'files3/S033R05.edf',
 'files3/S034R05.edf',
 'files3/S035R05.edf',
 'files3/S036R05.edf',
 'files3/S037R05.edf',
 'files3/S038R05.edf',
 'files3/S039R05.edf',
 'files3/S040R05.edf',
 'files3/S041R05.edf',
 'files3/S042R05.edf',
 'files3/S043R05.edf',
 'files3/S0

In [4]:
train_split=0.8

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(900000, 112425, 900000, 112425)

In [10]:
print(xtest.iloc[:,-1].values)

[ 1.01e-04  1.30e-05 -1.50e-05 ... -3.00e-05  1.00e-05  1.00e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000042,0.000014,0.000028,0.000021,0.000018,0.000058,-0.000033,0.000031,0.000055,0.000040,...,-0.000015,-0.000003,0.000044,-0.000002,-0.000017,-0.000032,-0.000032,0.000053,-0.000053,0.000021
899996,-0.000008,-0.000014,-0.000018,-0.000019,-0.000005,-0.000012,-0.000056,0.000001,0.000026,0.000005,...,-0.000020,-0.000024,0.000017,-0.000032,-0.000049,-0.000034,-0.000055,0.000032,-0.000056,0.000002
899997,0.000003,-0.000015,-0.000017,-0.000018,-0.000001,-0.000014,-0.000050,0.000010,0.000018,0.000002,...,-0.000001,-0.000029,0.000021,-0.000014,-0.000042,-0.000046,-0.000050,0.000022,-0.000082,-0.000003
899998,0.000063,0.000026,0.000021,0.000018,0.000032,-0.000003,-0.000011,0.000049,0.000055,0.000046,...,0.000014,0.000046,0.000100,0.000050,0.000015,-0.000002,0.000037,0.000114,-0.000044,0.000064


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/tmp/ipykernel_21844/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/tmp/ipykernel_21844/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/tmp/ipykernel_21844/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000042,0.000014,0.000028,0.000021,0.000018,0.000058,-0.000033,0.000031,0.000055,0.000040,...,-0.000015,-0.000003,0.000044,-0.000002,-0.000017,-0.000032,-0.000032,0.000053,-0.000053,0.000021
899996,-0.000008,-0.000014,-0.000018,-0.000019,-0.000005,-0.000012,-0.000056,0.000001,0.000026,0.000005,...,-0.000020,-0.000024,0.000017,-0.000032,-0.000049,-0.000034,-0.000055,0.000032,-0.000056,0.000002
899997,0.000003,-0.000015,-0.000017,-0.000018,-0.000001,-0.000014,-0.000050,0.000010,0.000018,0.000002,...,-0.000001,-0.000029,0.000021,-0.000014,-0.000042,-0.000046,-0.000050,0.000022,-0.000082,-0.000003
899998,0.000063,0.000026,0.000021,0.000018,0.000032,-0.000003,-0.000011,0.000049,0.000055,0.000046,...,0.000014,0.000046,0.000100,0.000050,0.000015,-0.000002,0.000037,0.000114,-0.000044,0.000064


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
model=XGBClassifier(n_estimators=500)
model.fit(x8,y8,early_stopping_rounds=10, eval_set=[(xv8, yv8)])
y_pred=model.predict(xt8)
print(classification_report(yt8,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:4.03149
[1]	validation_0-mlogloss:3.91394
[2]	validation_0-mlogloss:3.83347
[3]	validation_0-mlogloss:3.77260
[4]	validation_0-mlogloss:3.72437
[5]	validation_0-mlogloss:3.68302
[6]	validation_0-mlogloss:3.64872
[7]	validation_0-mlogloss:3.61775
[8]	validation_0-mlogloss:3.59197
[9]	validation_0-mlogloss:3.57038
[10]	validation_0-mlogloss:3.54968
[11]	validation_0-mlogloss:3.53214
[12]	validation_0-mlogloss:3.51541
[13]	validation_0-mlogloss:3.49913
[14]	validation_0-mlogloss:3.48363
[15]	validation_0-mlogloss:3.46972
[16]	validation_0-mlogloss:3.45697
[17]	validation_0-mlogloss:3.44542
[18]	validation_0-mlogloss:3.43528
[19]	validation_0-mlogloss:3.42168
[20]	validation_0-mlogloss:3.41323
[21]	validation_0-mlogloss:3.40196
[22]	validation_0-mlogloss:3.39286
[23]	validation_0-mlogloss:3.38379
[24]	validation_0-mlogloss:3.37387
[25]	validation_0-mlogloss:3.36519
[26]	validation_0-mlogloss:3.35823
[27]	validation_0-mlogloss:3.34968
[28]	validation_0-mlogloss:3.3

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
28125/28125 [==============================] - 19s 682us/step - loss: 2.7639 - val_loss: 2.8109
Epoch 2/10
28125/28125 [==============================] - 19s 670us/step - loss: 2.1651 - val_loss: 2.7504
Epoch 3/10
28125/28125 [==============================] - 19s 677us/step - loss: 2.0587 - val_loss: 2.8817
Epoch 4/10
28125/28125 [==============================] - 23s 822us/step - loss: 2.0073 - val_loss: 2.8386
Epoch 5/10
28125/28125 [==============================] - 23s 811us/step - loss: 1.9742 - val_loss: 2.8400
Epoch 6/10
28125/28125 [==============================] - 23s 828us/step - loss: 1.9503 - val_loss: 2.7953
Epoch 7/10
28125/28125 [==============================] - 22s 791us/step - loss: 1.9318 - val_loss: 2.8855
Epoch 8/10
28125/28125 [==============================] - 23s 806us/step - loss: 1.9164 - val_loss: 2.8617
Epoch 9/10
28125/28125 [==============================] - 23s 833us/step - loss: 1.9043 - val_loss: 2.8096
Epoch 10/10
28125/28125 [============

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

3514/3514 [==============================] - 1s 346us/step
              precision    recall  f1-score   support

           0       0.09      0.10      0.10      1499
           1       0.11      0.05      0.07      1499
           2       0.43      0.26      0.33      1499
           3       0.56      0.31      0.40      1499
           4       0.52      0.74      0.61      1499
           5       0.41      0.63      0.49      1499
           6       0.31      0.49      0.38      1499
           7       0.50      0.47      0.48      1499
           8       0.54      0.52      0.53      1499
           9       0.48      0.17      0.25      1499
          10       0.30      0.40      0.34      1499
          11       0.33      0.43      0.37      1499
          12       0.27      0.24      0.25      1499
          13       0.18      0.12      0.15      1499
          14       0.14      0.06      0.09      1499
          15       0.61      0.65      0.63      1499
          16       0.2

## 0-16

In [17]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [18]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/tmp/ipykernel_21844/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/tmp/ipykernel_21844/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/tmp/ipykernel_21844/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,0.000042,0.000014,0.000028,0.000021,0.000018,0.000058,-0.000033,0.000031,0.000055,0.000040,...,-0.000015,-0.000003,0.000044,-0.000002,-0.000017,-0.000032,-0.000032,0.000053,-0.000053,0.000021
899996,-0.000008,-0.000014,-0.000018,-0.000019,-0.000005,-0.000012,-0.000056,0.000001,0.000026,0.000005,...,-0.000020,-0.000024,0.000017,-0.000032,-0.000049,-0.000034,-0.000055,0.000032,-0.000056,0.000002
899997,0.000003,-0.000015,-0.000017,-0.000018,-0.000001,-0.000014,-0.000050,0.000010,0.000018,0.000002,...,-0.000001,-0.000029,0.000021,-0.000014,-0.000042,-0.000046,-0.000050,0.000022,-0.000082,-0.000003
899998,0.000063,0.000026,0.000021,0.000018,0.000032,-0.000003,-0.000011,0.000049,0.000055,0.000046,...,0.000014,0.000046,0.000100,0.000050,0.000015,-0.000002,0.000037,0.000114,-0.000044,0.000064


In [19]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [20]:
model2=XGBClassifier(n_estimators=500)
model2.fit(x16,y16,early_stopping_rounds=10, eval_set=[(xv16, yv16)])
y_pred=model2.predict(xt16)
print(classification_report(yt16,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.97048
[1]	validation_0-mlogloss:3.81442
[2]	validation_0-mlogloss:3.71154
[3]	validation_0-mlogloss:3.63355
[4]	validation_0-mlogloss:3.56809
[5]	validation_0-mlogloss:3.51260
[6]	validation_0-mlogloss:3.46377
[7]	validation_0-mlogloss:3.41870
[8]	validation_0-mlogloss:3.38130
[9]	validation_0-mlogloss:3.34891
[10]	validation_0-mlogloss:3.31777
[11]	validation_0-mlogloss:3.29204
[12]	validation_0-mlogloss:3.26906
[13]	validation_0-mlogloss:3.24562
[14]	validation_0-mlogloss:3.22182
[15]	validation_0-mlogloss:3.20076
[16]	validation_0-mlogloss:3.18285
[17]	validation_0-mlogloss:3.16361
[18]	validation_0-mlogloss:3.14614
[19]	validation_0-mlogloss:3.12971
[20]	validation_0-mlogloss:3.11554
[21]	validation_0-mlogloss:3.09928
[22]	validation_0-mlogloss:3.08313
[23]	validation_0-mlogloss:3.07136
[24]	validation_0-mlogloss:3.05768
[25]	validation_0-mlogloss:3.04453
[26]	validation_0-mlogloss:3.03198
[27]	validation_0-mlogloss:3.01974
[28]	validation_0-mlogloss:3.0

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
28125/28125 [==============================] - 25s 871us/step - loss: 2.0717 - val_loss: 2.4269
Epoch 2/10
28125/28125 [==============================] - 24s 843us/step - loss: 1.3700 - val_loss: 2.4978
Epoch 3/10
28125/28125 [==============================] - 24s 846us/step - loss: 1.2189 - val_loss: 2.4654
Epoch 4/10
28125/28125 [==============================] - 24s 843us/step - loss: 1.1444 - val_loss: 2.5653
Epoch 5/10
28125/28125 [==============================] - 24s 842us/step - loss: 1.0991 - val_loss: 2.4636
Epoch 6/10
28125/28125 [==============================] - 24s 851us/step - loss: 1.0644 - val_loss: 2.5869
Epoch 7/10
28125/28125 [==============================] - 24s 853us/step - loss: 1.0391 - val_loss: 2.5670
Epoch 8/10
28125/28125 [==============================] - 24s 848us/step - loss: 1.0161 - val_loss: 2.6999
Epoch 9/10
28125/28125 [==============================] - 24s 855us/step - loss: 0.9977 - val_loss: 2.5645
Epoch 10/10
28125/28125 [============

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

3514/3514 [==============================] - 1s 352us/step
              precision    recall  f1-score   support

           0       0.31      0.26      0.28      1499
           1       0.24      0.22      0.23      1499
           2       0.31      0.33      0.32      1499
           3       0.57      0.40      0.47      1499
           4       0.83      0.85      0.84      1499
           5       0.54      0.70      0.61      1499
           6       0.73      0.86      0.79      1499
           7       0.93      0.55      0.69      1499
           8       0.53      0.36      0.43      1499
           9       0.49      0.53      0.51      1499
          10       0.62      0.61      0.61      1499
          11       0.45      0.79      0.57      1499
          12       0.41      0.58      0.48      1499
          13       0.31      0.29      0.30      1499
          14       0.24      0.13      0.17      1499
          15       0.85      0.73      0.78      1499
          16       0.4

## 0-32

In [21]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [22]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/tmp/ipykernel_21844/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/tmp/ipykernel_21844/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/tmp/ipykernel_21844/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

In [23]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [24]:
model3=XGBClassifier(n_estimators=500)
model3.fit(x32,y32,early_stopping_rounds=10, eval_set=[(xv32, yv32)])
y_pred=model3.predict(xt32)
print(classification_report(yt32,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.78198
[1]	validation_0-mlogloss:3.56577
[2]	validation_0-mlogloss:3.41848
[3]	validation_0-mlogloss:3.30050
[4]	validation_0-mlogloss:3.21090
[5]	validation_0-mlogloss:3.12435
[6]	validation_0-mlogloss:3.05434
[7]	validation_0-mlogloss:2.99031
[8]	validation_0-mlogloss:2.93154
[9]	validation_0-mlogloss:2.88247
[10]	validation_0-mlogloss:2.83831
[11]	validation_0-mlogloss:2.79809
[12]	validation_0-mlogloss:2.75996
[13]	validation_0-mlogloss:2.72445
[14]	validation_0-mlogloss:2.69246
[15]	validation_0-mlogloss:2.66259
[16]	validation_0-mlogloss:2.63346
[17]	validation_0-mlogloss:2.60690
[18]	validation_0-mlogloss:2.57859
[19]	validation_0-mlogloss:2.55227
[20]	validation_0-mlogloss:2.53067
[21]	validation_0-mlogloss:2.50680
[22]	validation_0-mlogloss:2.48669
[23]	validation_0-mlogloss:2.46623
[24]	validation_0-mlogloss:2.44582
[25]	validation_0-mlogloss:2.42645
[26]	validation_0-mlogloss:2.40596
[27]	validation_0-mlogloss:2.38764
[28]	validation_0-mlogloss:2.3

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
28125/28125 [==============================] - 27s 942us/step - loss: 1.4267 - val_loss: 1.8285
Epoch 2/10
28125/28125 [==============================] - 26s 930us/step - loss: 0.6363 - val_loss: 1.6128
Epoch 3/10
28125/28125 [==============================] - 27s 944us/step - loss: 0.4793 - val_loss: 1.7520
Epoch 4/10
28125/28125 [==============================] - 26s 940us/step - loss: 0.4106 - val_loss: 1.6017
Epoch 5/10
28125/28125 [==============================] - 27s 953us/step - loss: 0.3693 - val_loss: 1.5204
Epoch 6/10
28125/28125 [==============================] - 27s 947us/step - loss: 0.3410 - val_loss: 1.4673
Epoch 7/10
28125/28125 [==============================] - 29s 1ms/step - loss: 0.3202 - val_loss: 1.6347
Epoch 8/10
28125/28125 [==============================] - 28s 979us/step - loss: 0.3021 - val_loss: 1.8332
Epoch 9/10
28125/28125 [==============================] - 27s 974us/step - loss: 0.2882 - val_loss: 1.5334
Epoch 10/10
28125/28125 [==============

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

3514/3514 [==============================] - 1s 370us/step
              precision    recall  f1-score   support

           0       0.72      0.61      0.66      1499
           1       0.69      0.44      0.54      1499
           2       0.70      0.72      0.71      1499
           3       0.65      0.44      0.52      1499
           4       0.91      0.94      0.92      1499
           5       0.96      0.87      0.91      1499
           6       0.93      0.93      0.93      1499
           7       0.96      0.79      0.87      1499
           8       0.58      0.63      0.60      1499
           9       0.78      0.66      0.71      1499
          10       0.93      0.96      0.95      1499
          11       0.87      0.98      0.93      1499
          12       0.64      0.87      0.74      1499
          13       0.70      0.74      0.72      1499
          14       0.57      0.65      0.60      1499
          15       0.97      0.86      0.91      1499
          16       0.6

## 0-64

In [25]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [26]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [27]:
model4=XGBClassifier(n_estimators=500)
model4.fit(x,y,early_stopping_rounds=10, eval_set=[(xv, yv)])
y_pred=model4.predict(xt)
print(classification_report(yt,y_pred))

/home/codespace/.python/current/lib/python3.10/site-packages/xgboost/sklearn.py:885: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:3.57980
[1]	validation_0-mlogloss:3.30102
[2]	validation_0-mlogloss:3.11511
[3]	validation_0-mlogloss:2.96973
[4]	validation_0-mlogloss:2.84915
[5]	validation_0-mlogloss:2.74975
[6]	validation_0-mlogloss:2.66319
[7]	validation_0-mlogloss:2.58677
[8]	validation_0-mlogloss:2.52542
[9]	validation_0-mlogloss:2.46079
[10]	validation_0-mlogloss:2.40635
[11]	validation_0-mlogloss:2.35461
[12]	validation_0-mlogloss:2.31139
[13]	validation_0-mlogloss:2.27145
[14]	validation_0-mlogloss:2.23162
[15]	validation_0-mlogloss:2.18988
[16]	validation_0-mlogloss:2.15652
[17]	validation_0-mlogloss:2.12579
[18]	validation_0-mlogloss:2.09738
[19]	validation_0-mlogloss:2.06662
[20]	validation_0-mlogloss:2.04146
[21]	validation_0-mlogloss:2.01777
[22]	validation_0-mlogloss:1.99128
[23]	validation_0-mlogloss:1.96807
[24]	validation_0-mlogloss:1.94619
[25]	validation_0-mlogloss:1.92318
[26]	validation_0-mlogloss:1.90136
[27]	validation_0-mlogloss:1.88164
[28]	validation_0-mlogloss:1.8

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(75, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
28125/28125 [==============================] - 28s 991us/step - loss: 1.0268 - val_loss: 2.0086
Epoch 2/10
28125/28125 [==============================] - 28s 982us/step - loss: 0.4124 - val_loss: 2.0323
Epoch 3/10
28125/28125 [==============================] - 27s 966us/step - loss: 0.2943 - val_loss: 1.7633
Epoch 4/10
28125/28125 [==============================] - 28s 995us/step - loss: 0.2326 - val_loss: 1.6273
Epoch 5/10
28125/28125 [==============================] - 28s 995us/step - loss: 0.1949 - val_loss: 1.5107
Epoch 6/10
28125/28125 [==============================] - 28s 988us/step - loss: 0.1687 - val_loss: 1.5476
Epoch 7/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.1511 - val_loss: 1.5377
Epoch 8/10
28125/28125 [==============================] - 29s 1ms/step - loss: 0.1364 - val_loss: 1.6553
Epoch 9/10
28125/28125 [==============================] - 28s 1ms/step - loss: 0.1251 - val_loss: 1.3805
Epoch 10/10
28125/28125 [==================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

3514/3514 [==============================] - 1s 385us/step
              precision    recall  f1-score   support

           0       0.83      0.70      0.76      1499
           1       0.93      0.81      0.86      1499
           2       0.82      0.87      0.84      1499
           3       0.91      0.83      0.87      1499
           4       1.00      0.98      0.99      1499
           5       0.88      0.94      0.90      1499
           6       0.88      0.98      0.93      1499
           7       0.91      0.89      0.90      1499
           8       0.75      0.84      0.79      1499
           9       0.70      0.61      0.65      1499
          10       0.87      0.71      0.78      1499
          11       0.96      0.94      0.95      1499
          12       0.88      0.95      0.91      1499
          13       0.89      0.85      0.87      1499
          14       0.62      0.91      0.74      1499
          15       0.93      0.99      0.96      1499
          16       0.9